# 작사가 인공지능 만들기

## Step 1. 필요 모듈 임포트 및 데이터 가져오기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob
import os
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 0
Examples:
 []


## Step 2. 데이터 정제

### 문장 전처리 방식

1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 start, 끝에는 end를 추가합니다
7. 괄호 제거 - 코러스 싫다(추가)    
8. 토큰 갯수 15개 이하로 조정(추가)    

In [2]:
#정제함수 만들기
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = re.sub(r"\(.\)", " ", sentence) # 7
    sentence = '<start> ' + sentence + ' <end>' # 6
    
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        (sentences) sentence ."))

<start> this is sample sentences sentence . <end>


## 정제 데이터 구축하기
소스문장은 start + sentence + end, 타겟문장은 sentence + end

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 105 : continue 
    
    # 정제를 하고 corpus list에 담기
    preprocessed_sentence = preprocess_sentence(sentence) # 들어온 한 문장에 위의 함수처리한걸 받는 인자
    corpus.append(preprocessed_sentence) #함수처리한 str값을 corpus 리스트에 집어넣기
        
# 정제된 결과를 100개만 확인
print(corpus[:10])
len(corpus) #문장 갯수




[]


0

한 단어 평균 길이 6, 15어절이면 15단어 + 공백 15개 = 15x6 + 15 = 105 정도로 산정하여 코퍼스 길이를 조절했다. 이렇게 하면 긴 가사 데이터때문에 발생하는 과도한 패딩을 막을 수 있다.

## 데이터 벡터화(텐서화) - str 에서 숫자로
데이터를 숫자로 변환하기. 이 과정을 벡터화(vectorize) 라 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 칭한다. 우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것.

In [4]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다

def tokenize(corpus): #corpus : 정제된 문장str 들의 리스트
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, #인덱스 최대치
        filters=' ', #필터 필요 없어서 비워둠
        oov_token="<unk>" # 최대치
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  #post는 뒤에다가 패딩 붙이기
    
    print("토크나이저 : ",tokenizer,"\n" ,tensor)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor.shape

ValueError: zero-size array to reduction operation maximum which has no identity

기존에 학습을 진행하려 헀을떄, 텐서의 크기가 12001x256x347로 매우 길었다. 아무래도 가사 중 한 줄이 매우 길어, 나머지 가사 데이터에도 과도한 패딩이 삽입된것으로 판단되었다. 하여 정제 데이터를 만들 때 15어절 이상 되는 문장들을 모두 절삭했다,

#### corpus tensor 모든 로우의 시작값이 2, 문장 종료값이 3, 나머지는 0인 이유?

인덱스 2,3이 start와 end라서 모든 코퍼스 내 원소의 시작은 2, 끝지점은 3, 7000단어까지 인덱싱을 하고도 남은 단어는 1(unk), 남은 길이는 패딩인 0이 된다.

In [ ]:
#토크나이저 내부 인덱스별 할당된 단어 모음 1~10까지만 보기
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

## 훈련데이터와 평가데이터 분리

tensor에서 마지막 토큰을 잘라내어 소스문장을 만들고(end보단 pad가 지워질 가능성 多), 첫번째 start를 잘래내서 타겟 문장을 만든다.
이렇게 함으로서 scr_input과 trg_input의 크기를 맞춘다.

사이킷 런 split함수를 이용해서 훈련용 데이터와 검증용 데이터를 나눈다  

In [ ]:
# 소스문장과 타겟문장으로 1차 분리
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    
print("텐서 길이: ",tensor.shape)
print("소스문장 길이: ",len(src_input[0])) #346 start + sentence + end + padding n개
print("타겟문장 길이: ",len(tgt_input[0])) #346 sentence + end + 패딩 n+1개 




# 각각의 데이터에서 훈련셋, 평가셋 나누기

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)

print("Source Train 길이:", enc_train.shape)
print("Target Train 길이:", dec_train.shape)

print("Source Test 길이:", enc_val.shape)
print("Target Test 길이:", dec_val.shape)

## tf.data.Dataset객체를 생성

tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성

In [ ]:
# 하이퍼파라미터 설정
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   
# tokenizer가 구축한 단어사전 내 12000개 + 0:<pad> = 12001개


# 1. 학습데이터셋 객체 만들기
dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)) #각 원소별로 자르기
dataset_train = dataset_train.shuffle(BUFFER_SIZE)                    # 해당 길이에서 랜덤으로 원소 나열
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True) # 해당 값만큼의 연속된 원소들 묶음

#2. 검증데이터셋 객체 만들기
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)) #각 원소별로 자르기
dataset_val = dataset_val.shuffle(BUFFER_SIZE)                    # 해당 길이에서 랜덤으로 원소 나열
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True) # 해당 값만큼의 연속된 원소들 묶음

print(dataset_train)
print(dataset_val)


만약 데이터가 같은 크기(차원)을 가졌다면, drop-remainder를 True(나머지 원소 버림)

배치 사이즈로 데이터를 나누다 보면, 마지막에 배치 사이즈보다 작아서 안나눠지는 원소들이 있다. 이걸 안없애면마지막 배치는 다른 배치들과 크기가 안맞아서 오류가 날수도. 배치사이즈가 데이터 길이와 정확히 나누어 떨어지지 않을떈 True로 바꿔주자.

## LSTM 모델설명

우리 입력 텐서에는 단어 사전의 인덱스가 들어 있다. Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 준다. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용된다.

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__() #얘는 무슨 초기화값을 상속받는것인가
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size) #단어책 사이즈로 줄여줘야 다음에 무슨 단어를 낼지에 대한 각 클래스별 수치 나옴
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024 #hidden state의 차원 수 : 판단 횟수? or 판단 다양성? 판단하는 두뇌 수?
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) #모델 정의
model

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset_train.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

shape=(256, 34, 12001)
12001 - dense layer 출력 차원수 : 다음에 무슨 단어가 올 확률이 높은가?

256 - 배치사이즈, 256개의 문장 가져옴 take(1) 이므로 배치 1번

34 - tf.keras.layers.LSTM(hidden_size, return_sequences=True)에서 return_sequences=True와 연관있다. 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스 출력하라는 의미 return_sequences=False 면 1로 출력됨

### 모델 구조 확인

In [ ]:
model.summary()
tf.test.is_gpu_available() #gpu 사용하고 있는지?

## 모델 학습시키기

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_train, epochs=10)

## 모델 사용해 보기

In [ ]:
#단어 
#생성기
def generate_text(model, tokenizer, init_sentence="<start>", max_len=30):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

#### While문의 역할
해당 모델을 평가하기 위한 평가지표인 테스트 셋이 존재하지 않는다. 또한, 소스문장도 따로 존재하지 않는다. 그럼 평가척도는?

while문은 RNN의 구조상 필요한 형식이다. 루프를 한번 돌면 start 뒤에 가장 높은 확률의 단어 인덱스를 삽입한다.

아래의 for문은 해당 인덱스에 할당된 단어 str을 generated str값 안에다가 집어넣어 문장을 완성한다.

## 단어 생성!

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

## 회고

### 힘들었던 부분
가장 힘들었던 부분은 데이터 정제와 인코딩을 통해 텐서화가 되며, 현재 데이터의 차원을 계속 인지하는 부분이였다. 데이터의 크기와 모양이 맞지 않아 그걸 맞추느라 고생했고, 두번째는 과연 어떻게 15 어절에서 토큰을 끊어낼 수 있는가 였다. 처음에는 텐서화시키는 부분에서 절삭을 하려했다. 다시말해, 패딩이 들어가는 부분에서 처리를 하려 했는데, 그렇게 하니 긴 문장에서 end값이 삭제되어 버린다는 것을 깨달았다. 결국 데이터 전처리 부분, 즉 정규표현식을 이용한 str값 처리 + 예외조항에서 처리해 줘야하는 거였다. start값과 end값을 삽입하기 이전 데이터에서 말이다. 단어 어절 별로 끊는 방법을 물색해 봤으나, 결국 찾지 못해 아쉬운대로 코퍼스의 길이값을 이용해 끊어내기로 했다. 한 단어의 평균 길이를 6정도로 산정하고, (기준은 played, danced, killed 등 동사 + ed 등의 단어가 자주 나오는 듯 하여 그렇게 구성함. 훨씬 긴 단어도 많았지만, i나 he 등과  같이 짧은 길이 단어도 존재하니, 어느정도 참작이 될 거라 판단했다.), 15어절이니 띄어쓰기 15칸을 포함해 길이 105를 산정했다. 105보다 긴 길이의 코퍼스는 생각보다 많지 않았고, 이를 통해 이상치들을 삭제할 수 있었다. 더 정확한 방법을 찾지 못해 사용한 임시방편이였으나, 결괏값이 좋았으므로 나쁘지 않은 접근방식으로 생각된다.

### 느낀점
저번 프로젝트와는 다르게 코드가 나오면 파라미터는 무엇이 있는지 하나하나 차근차근 확인해 보며 진행했다. 시간은 수십배 더 걸렸으나, 자잘한 오류나 변수 오판 등의 실수를 상당히 감소시켰다. 마지막으로, 문제 해결을 위해 코딩만 잘하면 되는 게 아니라 어떤식으로 접근할지 생각하는지도 굉장히 중요하다는 걸 깨달았다. 접근 방식을 달리하니, 어렵게 구현할 문제도 굉장히 쉬워지곤 했다. 궁금한 점은 시키는대로 트레이닝 셋과 테스트 셋을 나눴는데, 테스트셋이 어디에 들어가는지를 인지하지 못했다. 아직 모델 학습에 관한 이해도가 부족한 것으로 보여 모델 학습과 관련한 코드를 더 뜯어보고자 다짐한다.